In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import gc

from argparse import Namespace
from mimetypes import suffix_map
from pathlib import Path
import numpy as np
import yaml
import torch

from pytorch_lightning.trainer import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from kolmopy.dataloaders import TurboFlowDataModule
import kolmopy.utils.viz_utils as viz
import kolmopy.utils.phy_utils as phy
import turboflow.utils.torch_utils as tch
from turboflow.utils.torch_utils import to_np

import matplotlib.pyplot as plt

In [ ]:
exp_name = 'moMFN'
base_dir = Path('/','home','dicarlo_d','Documents','Code','TurboSuperResultion','recipes',exp_name)
data_dir = Path('/','home','dicarlo_d','Documents','Code','TurboSuperResultion','.cache', 'Synth2D_0003.hdf5')
mdl_dir = base_dir / Path('models')
fig_dir = base_dir / Path('figures')
res_dir = base_dir / Path('results')

print(base_dir.exists())
print(data_dir.exists())

print(fig_dir.exists())
print(res_dir.exists())
print(res_dir.exists())

In [ ]:
seed = 666
seed_everything(seed, workers=True)

batch_size = {
    'train' : 64*64
,   'val'   : 64*64
,   'test'  : 64*64
}

dx = {
    'train' : 4
,   'val'   : 2
,   'test'  : 1
}
dt = {
    'train' : 4
,   'val'   : 2
,   'test'  : 1
}
time_idxs = np.array([8, 16, 24, 31])

dataset_name = 'Synth2D'
dm_test = TurboFlowDataModule(
    dataset=dataset_name, 
    data_dir=data_dir,
    time_idx=time_idxs,
    add_vorticity='True',

    train_batch_size=batch_size['train'],
    val_batch_size=batch_size['val'],
    test_batch_size=batch_size['test'],

    train_downsampling_space=dx['train'],
    val_downsampling_space=dx['val'],
    test_downsampling_space=dx['test'],

    train_downsampling_time=dt['train'],
    val_downsampling_time=dt['val'],
    test_downsampling_time=dt['test'],

    train_shuffle=False,
    val_shuffle=False,
    test_shuffle=False,
    num_workers=1
)

dm_test.setup()
# datasets = [dm_train.train_dataset, dm_train.val_dataset]
datasets = [dm_test.train_dataset, dm_test.val_dataset, dm_test.test_dataset]
for dataset in datasets:
    X, y, extra = dataset[:]
    print(dataset.ax_t)
    print(X.shape, y.shape, dataset.img_shape, dataset.vars_shape_img, dataset.fields_shape_img)

## Pre-Train: one layer at the time

In [ ]:
from turboflow.models.phyrff import plBACON
models_dir = '../recipes/moMFN/models/'
pre_train_best_model_name = 'Synth2D_0003_moMFN_2x32x32-pre_train-epoch=69-val_loss=0.00000.ckpt'
pre_train_best_model_path = models_dir + pre_train_best_model_name
model = plBACON.load_from_checkpoint(pre_train_best_model_path)

In [ ]:
g0 = model.mfn.g0
g1 = model.mfn.g1
g2 = model.mfn.g2
g3 = model.mfn.g3
g4 = model.mfn.g4
g5 = model.mfn.g5

plt.figure(figsize=(20,20))
for g in [g0, g1, g2, g3, g4, g5]:
    Bm = g.Bmin
    BM = Bm + g.Bwidth
    w = g.linear.weight.data.detach().cpu().numpy()
    plt.scatter(w[:,-2], w[:,-1], alpha=0.5, label=f'{Bm}-{BM}')

plt.legend()
plt.show()

In [ ]:
y1 = model.mfn.y1
y2 = model.mfn.y2
y3 = model.mfn.y3
y4 = model.mfn.y4
y5 = model.mfn.y5

for k, layer in enumerate([y1, y2, y3, y4, y5]):
    w = layer.weight.data.detach().cpu().numpy()
    plt.hist(w.flatten(), alpha=0.7, histtype='step', label=f'{k}')

plt.legend()
plt.show()

In [ ]:
seed = 666
seed_everything(seed, workers=True)

batch_size = {
    'train' : 64*64
,   'val'   : 64*64
,   'test'  : 64*64
}

dx = {
    'train' : 4
,   'val'   : 2
,   'test'  : 1
}
dt = {
    'train' : 4
,   'val'   : 4
,   'test'  : 4
}
time_idxs = np.arange(4)

dataset_name = 'Synth2D'
dm_test = TurboFlowDataModule(
    dataset=dataset_name, 
    data_dir=data_dir,
    time_idx=time_idxs,
    add_vorticity=True,

    train_batch_size=batch_size['train'],
    val_batch_size=batch_size['val'],
    test_batch_size=batch_size['test'],

    train_downsampling_space=dx['train'],
    val_downsampling_space=dx['val'],
    test_downsampling_space=dx['test'],

    train_downsampling_time=dt['train'],
    val_downsampling_time=dt['val'],
    test_downsampling_time=dt['test'],

    train_shuffle=False,
    val_shuffle=False,
    test_shuffle=False,
    num_workers=1
)

dm_test.setup()
# datasets = [dm_train.train_dataset, dm_train.val_dataset]
datasets = [dm_test.train_dataset, dm_test.val_dataset, dm_test.test_dataset]
for dataset in datasets:
    X, y, extra = dataset[:]
    print(dataset.ax_t)
    print(X.shape, y.shape, dataset.img_shape, dataset.vars_shape_img, dataset.fields_shape_img)

In [ ]:
xy, uv, extra = dataset[:]

vshape = dataset.vars_shape_img
fshape = dataset.fields_shape_img
print(fshape)

In [ ]:
xy1 = xy.reshape(*vshape)[0,...].view(-1,vshape[-1])
uv1 = uv.reshape(*fshape)[0,...].view(-1,fshape[-1])

In [ ]:
xy2 = xy.reshape(*vshape)[1,...].view(-1,vshape[-1])
uv2 = uv.reshape(*fshape)[1,...].view(-1,fshape[-1])

In [ ]:
XY = xy.reshape(*vshape)[0,:,:,1:].numpy()
UV1 = uv1.reshape(fshape[1:]).numpy()
UV2 = uv2.reshape(fshape[1:]).numpy()

plt.figure(figsize=(20,20))
plt.subplot(121)
plt.imshow(UV1[:,:,0])
plt.subplot(122)
plt.imshow(UV1[:,:,1])
plt.show()

In [ ]:
xy.requires_grad_(True)
Nt, Nx, Ny, D = fshape

uv_pred  = model(xy, out=4)
print(uv_pred.shape)

UV_ = uv_pred.reshape(Nt, Nx, Ny, 2)
plt.imshow(tch.to_np(UV_[0,:,:,0]))
plt.show()

In [ ]:
W1 = phy.compute_vorticity(XY, UV1)/256
W2 = phy.compute_vorticity(XY, UV2)

plt.subplot(121)
plt.imshow(W1[8:-8,8:-8])
plt.colorbar()
plt.subplot(122)
plt.imshow(W2)
plt.show()

In [ ]:
W_t = (W2 - W1)/(1/30)
print(np.mean(W_t))
plt.imshow(W_t[8:-8,8:-8])
plt.colorbar()
plt.show()

In [ ]:
[W_x, W_y] = phy.my_grad([W1],[XY[:,0,0], XY[0,:,1]], indexing='ij')[0]

W_a = W_x * UV1[:,:,0] + W_y * UV1[:,:,1]

plt.imshow(W_a[8:-8,8:-8])
plt.colorbar()
plt.show()

In [ ]:
[W_xx, W_xy] = phy.my_grad([W_x],[XY[:,0,0],XY[0,:,1]], indexing='ij')[0]
[W_yx, W_yy] = phy.my_grad([W_y],[XY[:,0,0],XY[0,:,1]], indexing='ij')[0]

W_d = W_xx + W_yy

plt.imshow(W_d[8:-8,8:-8])
plt.colorbar()
plt.show()

In [ ]:
PDE = W_t - W_a #- W_d * dataset.diffusion

plt.imshow(PDE[8:-8,8:-8])
plt.colorbar()
plt.show()

In [ ]:
t = torch.linspace( 0., 1., 15)
print(t)

In [ ]:
# for dataset in [dm_test.train_dataset, dm_test.val_dataset, dm_test.test_dataset]:

for dataset in [dm_test.test_dataset]:
    xy, uv, extra = dataset[:]

    vshape = dataset.vars_shape_img
    fshape = dataset.fields_shape_img


    t = 8
    xy = xy.reshape(*vshape)[t,...].view(-1,vshape[-1])
    uv = uv.reshape(*fshape)[t,...].view(-1,fshape[-1])
    print(xy.shape)
    print(uv.shape)
    
    print(xy)

    fshape = dataset.fields_shape_img
    Nt, Rx, Ry, D = fshape
    vshape = dataset.vars_shape_img

    xy.requires_grad_(True)
    uv_pred = model(xy)[0]

    plt.subplot(121)
    plt.imshow(to_np(uv.reshape(*fshape[1:])[:,:,0]))
    plt.subplot(122)
    plt.imshow(to_np(uv_pred.reshape(Nt, Rx, Rx, 2)[:,:,0]))
    plt.show()


    u, v = torch.split(uv_pred, 1, -1)

    du = tch.diff(u, xy)
    u_t, u_x, u_y = du.split(1, -1)

    dv = tch.diff(v, xy)
    v_t, v_x, v_y = dv.split(1, -1)

    f = to_np(uv.reshape(*fshape[1:])[:,:,0])
    x = to_np(xy.reshape(*vshape[1:])[:,:,1])
    
    div = u_x + v_y
    plt.imshow(to_np(div.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

    u_x_diff = phy.my_grad([f], [x[:,0]], indexing='ij')[0][0]

    plt.subplot(121)
    plt.imshow(u_x_diff)
    plt.subplot(122)
    plt.imshow(to_np(u_x.reshape(*fshape[1:3])))
    plt.show()

    w = v_x - u_y

    w_diff = phy.compute_vorticity(to_np(xy.reshape(*vshape[1:])[:,:,1:]),
                                   to_np(uv.reshape(*fshape[1:])), indexing='ij')

    plt.subplot(121)
    plt.imshow(w_diff)
    plt.subplot(122)
    plt.imshow(to_np(w.reshape(*fshape[1:3])))
    plt.show()

## Fine Tuning: all layers

In [ ]:
from turboflow.models.phyrff import plBACON

best_model_path='/home/dicarlo_d/Documents/Code/TurboSuperResultion/recipes/moMFN/models/Synth2D_0001_moMFN_15x32x32-fine_tune-epoch=479-val_loss=0.00000.ckpt'
model = plBACON.load_from_checkpoint(best_model_path)
model.eval()

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.to(device)

In [ ]:
g0 = model.mfn.g0
g1 = model.mfn.g1
g2 = model.mfn.g2
g3 = model.mfn.g3
g4 = model.mfn.g4
g5 = model.mfn.g5

plt.figure(figsize=(20,20))
for g in [g0, g1, g2, g3, g4, g5]:
    Bm = g.Bmin
    BM = Bm + g.Bwidth
    w = g.linear.weight.data.detach().cpu().numpy()
    plt.scatter(w[:,-2], w[:,-1], alpha=0.5, label=f'{Bm}-{BM}')

plt.legend()
plt.show()

In [ ]:
for dataset in [dm_test.train_dataset, dm_test.val_dataset, dm_test.test_dataset]:

    xy, uv, extra = dataset[:]
#     xy = xy.to(device)
#     uv = uv.to(device)

    vshape = dataset.vars_shape_img
    fshape = dataset.fields_shape_img


    t = 2
    xy = xy.reshape(*vshape)[t,...].view(-1,vshape[-1])
    uv = uv.reshape(*fshape)[t,...].view(-1,fshape[-1])
    print(xy.shape)
    print(uv.shape)

    fshape = dataset.fields_shape_img
    vshape = dataset.vars_shape_img

    xy.requires_grad_(True)
    uv_pred = model.mfn(xy)[0]

    plt.subplot(121)
    plt.imshow(to_np(uv.reshape(*fshape[1:])[:,:,0]))
    plt.subplot(122)
    plt.imshow(to_np(uv_pred.reshape(*fshape[1:])[:,:,0]))
    plt.show()

    u, v = torch.split(uv_pred, 1, -1)

    du = tch.diff(u, xy)
    u_t, u_x, u_y = du.split(1, -1)

    dv = tch.diff(v, xy)
    v_t, v_x, v_y = dv.split(1, -1)

    f = to_np(uv.reshape(*fshape[1:])[:,:,0])
    x = to_np(xy.reshape(*vshape[1:])[:,:,1])

    div = u_x + v_y
    plt.imshow(to_np(div.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

    u_x_diff = phy.my_grad([f], [x[:,0]], indexing='ij')[0][0]
    
    u_x_diff = phy.my_grad([f], [x[:,0]], indexing='ij')[0][0]

    plt.subplot(121)
    plt.imshow(u_x_diff)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(to_np(u_x.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

    w = v_x - u_y

    w_diff = phy.compute_vorticity(to_np(xy.reshape(*vshape[1:])[:,:,1:]),
                                   to_np(uv.reshape(*fshape[1:])), indexing='ij')

    plt.subplot(121)
    plt.imshow(w_diff)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(to_np(w.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()
    


## PDE Tuning: all layers

In [ ]:
from turboflow.models.phyrff import plBACON

best_model_path='/home/dicarlo_d/Documents/Code/TurboSuperResultion/recipes/moMFN/models/Synth2D_moMFN_31x32x32-pde_tune-epoch=79-val_loss=0.00000.ckpt'
model = plBACON.load_from_checkpoint(best_model_path)
model.eval()

In [ ]:
for dataset in [dm_test.train_dataset, dm_test.val_dataset, dm_test.test_dataset]:

    xy, uv, extra = dataset[:]
#     xy = xy.to(device)
#     uv = uv.to(device)

    vshape = dataset.vars_shape_img
    fshape = dataset.fields_shape_img


    t = 0
    xy = xy.reshape(*vshape)[t,...].view(-1,vshape[-1])
    uv = uv.reshape(*fshape)[t,...].view(-1,fshape[-1])
    print(xy.shape)
    print(uv.shape)

    fshape = dataset.fields_shape_img
    vshape = dataset.vars_shape_img

    xy.requires_grad_(True)
    uv_pred = model.mfn(xy)[0]

    plt.subplot(121)
    plt.imshow(to_np(uv.reshape(*fshape[1:])[:,:,0]))
    plt.subplot(122)
    plt.imshow(to_np(uv_pred.reshape(*fshape[1:])[:,:,0]))
    plt.show()


    u, v = torch.split(uv_pred, 1, -1)

    du = tch.diff(u, xy)
    u_t, u_x, u_y = du.split(1, -1)

    dv = tch.diff(v, xy)
    v_t, v_x, v_y = dv.split(1, -1)

    f = to_np(uv.reshape(*fshape[1:])[:,:,0])
    x = to_np(xy.reshape(*vshape[1:])[:,:,1])

    div = u_x + v_y
    plt.imshow(to_np(div.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

    u_x_diff = phy.my_grad([f], [x[:,0]], indexing='ij')[0][0]

    
    u_x_diff = phy.my_grad([f], [x[:,0]], indexing='ij')[0][0]

    plt.subplot(121)
    plt.imshow(u_x_diff)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(to_np(u_x.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

    w = v_x - u_y

    w_diff = phy.compute_vorticity(to_np(xy.reshape(*vshape[1:])[:,:,1:]),
                                   to_np(uv.reshape(*fshape[1:])), indexing='ij')

    plt.subplot(121)
    plt.imshow(w_diff)
    plt.colorbar()
    plt.subplot(122)
    plt.imshow(to_np(w.reshape(*fshape[1:3])))
    plt.colorbar()
    plt.show()

In [ ]:
g1 = model.mfn.l1
g2 = model.mfn.l2
g3 = model.mfn.l3
g4 = model.mfn.l4

for k, g in enumerate([g1, g2, g3, g4]):
    w = g.weight.data.detach().cpu().numpy()
    plt.hist(w.flatten(), alpha=0.7, histtype='step', label=f'{k}')

plt.legend()
# plt.xlim([-0.1, 0.1])
plt.show()

In [ ]:
dataset = dm_train.val_dataset
xy, uv, extra = dataset[:]
fshape = dataset.fields_shape_img
vshape = dataset.vars_shape_img

xy.requires_grad_(True)
uv_pred = model.mfn(xy)[0]

from turboflow.utils import torch_utils as tch

u, v = torch.split(uv_pred, 1, -1)
print(u.shape)

du = tch.diff(u, xy)
u_t, u_x, u_y = du.split(1, -1)

dv = tch.diff(v, xy)
v_t, v_x, v_y = dv.split(1, -1)

plt.imshow(u.detach().numpy().reshape(*fshape[:3])[0,:,:])
plt.show()
plt.imshow(u_x.detach().numpy().reshape(*fshape[:3])[0,:,:])
plt.show()

u_x_diff = np.diff(u.detach().numpy().reshape(*fshape[:3])[0,32,:])

plt.plot(u.detach().numpy().reshape(*fshape[:3])[0,32,:])
plt.plot(u_x.detach().numpy().reshape(*fshape[:3])[0,32,:])
plt.plot(u_x_diff)
plt.show()

In [ ]:
print(vshape)
print(fshape)
w = v_x - u_y

plt.imshow(w.detach().numpy().reshape(*fshape[:3])[1,:,:])

In [ ]:
dw = tch.diff(w, xy)
w_t, w_x, w_y = dw.split(1, -1)
ddw = tch.diff(dw, xy)
w_tt, w_xx, w_yy = ddw.split(1, -1)

# spatial change of vorticity / unit volume (u dot Nabla)w
w_spatial = u*w_x + v*w_y
# diffusion of vorticity / univ volume (nu Laplacian w)
diffusion = 0 # 1/self.Re
w_diffusion = diffusion*(w_xx + w_yy)

In [ ]:
plt.imshow(w_spatial.detach().numpy().reshape(*fshape[:3])[1,:,:])

In [ ]:
plt.imshow(w_t.detach().numpy().reshape(*fshape[:3])[1,:,:])

In [ ]:
plt.imshow(torch.abs(w_t + w_spatial).detach().numpy().reshape(*fshape[:3])[1,:,:])
plt.colorbar()

In [ ]:
from turboflow.utils import phy_utils as phy

In [ ]:
t = 0

UV = uv.detach().numpy().reshape(*fshape)
XY = xy.detach().numpy().reshape(*vshape)
print(UV.shape)
print(XY.shape)

plt.figure(figsize=(15,4))
plt.subplot(151)
plt.imshow(XY[t,:,:,0])
plt.subplot(152)
plt.imshow(XY[t,:,:,1])
plt.subplot(153)
plt.imshow(XY[t,:,:,2])
plt.subplot(154)
plt.imshow(UV[t,:,:,0])
plt.subplot(155)
plt.imshow(UV[t,:,:,1])
plt.tight_layout()
plt.show()

[[Udx, Udy], [Vdx, Vdy]] = phy.my_grad([UV[t,:,:,0], UV[t,:,:,1]], [XY[t,:,0,1], XY[t,0,:,2]])

plt.figure(figsize=(12,5))
plt.subplot(141)
plt.imshow(Udx)
plt.subplot(142)
plt.imshow(Udy)
plt.subplot(143)
plt.imshow(Vdx)
plt.subplot(144)
plt.imshow(Vdy)
plt.tight_layout()
plt.show()

W = Vdx - Udy

plt.figure(figsize=(5,5))
plt.imshow(W)
plt.show()

In [ ]:
t=0
[[Udx, Udy], [Vdx, Vdy]] = phy.my_grad([UV[t,:,:,0], UV[t,:,:,1]], [XY[t,:,0,1], XY[t,0,:,2]])
W0 = Vdx - Udy
t=1
[[Udx, Udy], [Vdx, Vdy]] = phy.my_grad([UV[t,:,:,0], UV[t,:,:,1]], [XY[t,:,0,1], XY[t,0,:,2]])
W1 = Vdx - Udy
t=2
[[Udx, Udy], [Vdx, Vdy]] = phy.my_grad([UV[t,:,:,0], UV[t,:,:,1]], [XY[t,:,0,1], XY[t,0,:,2]])
W2 = Vdx - Udy

Wdt = (W1-W0)/4 + (W2-W1)/4
plt.imshow(Wdt)
plt.colorbar()
plt.show()

In [ ]:
t=1
[[Udx, Udy], [Vdx, Vdy]] = phy.my_grad([UV[t,:,:,0], UV[t,:,:,1]], [XY[t,:,0,1], XY[t,0,:,2]])
W1 = Vdx - Udy

[[Wdx, Wdy]] = phy.my_grad([W1], [XY[t,:,0,1], XY[t,0,:,2]])
Wspat = UV[t,:,:,0]*Wdx + UV[t,:,:,1]*Wdy
plt.imshow(Wspat)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(Wdt/np.linalg.norm(Wdt) + Wspat/np.linalg.norm(Wspat))
plt.colorbar()